In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import fill
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.metrics import precision_recall_curve, average_precision_score, classification_report, confusion_matrix, roc_auc_score,auc
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import hdbscan
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mutual_info_score

# Data

In [2]:


%matplotlib inline

# Optional: wider display for DataFrame heads
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)

# Path to your already-cleaned CSV (change if needed)
CSV_PATH = "census_clean.csv"

# Load the cleaned dataset (no file is written in this notebook)
df = pd.read_csv(CSV_PATH)

print(df.shape)
df.head()

(196294, 42)


,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,race,hispanic_origin,sex,member_of_a_labor_union,reason_for_unemployment,full_or_part_time_employment_stat,capital_gains,capital_losses,dividends_from_stocks,tax_filer_stat,region_of_previous_residence,state_of_previous_residence,detailed_household_and_family_stat,detailed_household_summary_in_household,weight,migration_code-change_in_msa,migration_code-change_in_reg,migration_code-move_within_reg,live_in_this_house_1_year_ago,migration_prev_res_in_sunbelt,num_persons_worked_for_employer,family_members_under_18,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,fill_inc_questionnaire_for_veteran's_admin,veterans_benefits,weeks_worked_in_year,year,label
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


# Feature

In [3]:
# --- 1) Feature builder: column dropping, value normalization, derived features ---


class FeatureBuilder(BaseEstimator, TransformerMixin):
    def __init__(self):
        # columns we plan to drop (if present)
        self.drop_cols_ = [
            "year", "weight",  # survey-only
            "state_of_previous_residence",            # keep region instead
            "country_of_birth_father", "country_of_birth_mother"  # keep citizenship or self
        ]
        # pick one of the household pair if both exist
        self.household_pair_ = [
            "detailed_household_and_family_stat",
            "detailed_household_summary_in_household"
        ]
        # strings that mean missing/NA-ish in this dataset
        self.missing_tokens_ = ["Not in universe", "?", " Unknown", "unknown", "  ?"]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()

        # 1) unify missing-like tokens (categoricals)
        obj_cols = df.select_dtypes(include=["object", "category"]).columns
        for c in obj_cols:
            df[c] = (
                df[c].astype(str).str.strip()     
                    .replace(self.missing_tokens_, "Missing")
            )

        # 2) if region is missing but state exists, create region from state (BEFORE dropping state)
        if "region_of_previous_residence" not in df.columns and "state_of_previous_residence" in df.columns:
            # Census 4-region mapping by full state name
            northeast = {
                "Connecticut","Maine","Massachusetts","New Hampshire","Rhode Island","Vermont",
                "New Jersey","New York","Pennsylvania"
            }
            midwest = {
                "Illinois","Indiana","Michigan","Ohio","Wisconsin",
                "Iowa","Kansas","Minnesota","Missouri","Nebraska","North Dakota","South Dakota"
            }
            south = {
                "Delaware","District of Columbia","Florida","Georgia","Maryland","North Carolina","South Carolina","Virginia","West Virginia",
                "Alabama","Kentucky","Mississippi","Tennessee",
                "Arkansas","Louisiana","Oklahoma","Texas"
            }
            west = {
                "Arizona","Colorado","Idaho","Montana","Nevada","New Mexico","Utah","Wyoming",
                "Alaska","California","Hawaii","Oregon","Washington"
            }
            def state_to_region(s):
                s = str(s)
                if s in ("Missing", "Not in universe", "?", "unknown", "  ?"):
                    return "Missing"
                if s in northeast: return "Northeast"
                if s in midwest:   return "Midwest"
                if s in south:     return "South"
                if s in west:      return "West"
                # for territories / foreign / odd tokens
                if "puerto" in s.lower() or "guam" in s.lower() or "outlying" in s.lower():
                    return "Other"
                return "Other"
            df["region_of_previous_residence"] = df["state_of_previous_residence"].map(state_to_region)

        # 3) drop redundant/low-value columns (NOW after the mapping above)
        drop_exist = [c for c in self.drop_cols_ if c in df.columns]
        df = df.drop(columns=drop_exist, errors="ignore")

        # 4) if both household columns exist, keep the first one and drop the other
        both_have = [c for c in self.household_pair_ if c in df.columns]
        if len(both_have) == 2:
            df = df.drop(columns=[both_have[1]])

        # 5) derived features
        if "wage_per_hour" in df.columns:
            df["hourly_worker"] = (df["wage_per_hour"] > 0).astype(int)
            df["log_wage_per_hour"] = np.where(df["wage_per_hour"] > 0, np.log1p(df["wage_per_hour"]), 0.0)

        for col in ["capital_gains", "capital_losses", "dividends_from_stocks"]:
            if col in df.columns:
                df[f"has_{col}"] = (df[col] > 0).astype(int)
                df[f"log_{col}"] = np.where(df[col] > 0, np.log1p(df[col]), 0.0)

        if "weeks_worked_in_year" in df.columns:
            df["full_year_worker"] = (df["weeks_worked_in_year"] >= 50).astype(int)
            df["no_work"] = (df["weeks_worked_in_year"] == 0).astype(int)

        # 6) mobility flag (deterministic mapping based on your actual categories)

        move_signals = []

        # live_in_this_house_1_year_ago: Yes -> non-mover(0), No -> mover(1),
        # "Not in universe under 1 year old" -> unknown (NaN)
        if "live_in_this_house_1_year_ago" in df.columns:
            s = df["live_in_this_house_1_year_ago"].astype(str)
            map_house = {
                "Yes": 0,
                "No": 1,
                "Not in universe under 1 year old": np.nan,
                "Missing": np.nan,   # in case it was normalized earlier
            }
            move_signals.append(s.map(map_house))

        # migration_code-change_in_msa
        # Nonmover -> 0; any to/from MSA/NonMSA/Abroad -> 1; Not identifiable/Missing -> NaN
        if "migration_code-change_in_msa" in df.columns:
            s = df["migration_code-change_in_msa"].astype(str)
            map_msa = {
                "Nonmover": 0,
                "MSA to MSA": 1,
                "NonMSA to nonMSA": 1,
                "MSA to nonMSA": 1,
                "NonMSA to MSA": 1,
                "Abroad to MSA": 1,
                "Abroad to nonMSA": 1,
                "Not identifiable": np.nan,
                "Not in universe": np.nan,  # if normalization didn't run
                "Missing": np.nan,
            }
            move_signals.append(s.map(map_msa))

        # migration_code-change_in_reg
        # "Same county"/"Nonmover" -> 0; any "Different *" or "Abroad" -> 1; Not in universe/Missing -> NaN
        if "migration_code-change_in_reg" in df.columns:
            s = df["migration_code-change_in_reg"].astype(str)
            map_reg = {
                "Nonmover": 0,
                "Same county": 0,
                "Different region": 1,
                "Different county same state": 1,
                "Different division same region": 1,
                "Different state same division": 1,
                "Abroad": 1,
                "Not in universe": np.nan,
                "Missing": np.nan,
            }
            move_signals.append(s.map(map_reg))

        # migration_code-move_within_reg
        # "Same county"/"Nonmover" -> 0; any "Different state ..." or "Different county same state"/"Abroad" -> 1
        if "migration_code-move_within_reg" in df.columns:
            s = df["migration_code-move_within_reg"].astype(str)
            map_within = {
                "Nonmover": 0,
                "Same county": 0,
                "Different county same state": 1,
                "Different state in South": 1,
                "Different state in Northeast": 1,
                "Different state in Midwest": 1,
                "Different state in West": 1,
                "Abroad": 1,
                "Not in universe": np.nan,
                "Missing": np.nan,
            }
            move_signals.append(s.map(map_within))

        #  migration_prev_res_in_sunbelt is not a direct move indicator, so we ignore here.

        # Aggregate rule per row:
        # if any signal == 1 -> mover (1)
        # elif any signal == 0 -> non-mover (0)
        # else -> unknown (NaN)  (SimpleImputer in your numeric pipeline will fill this, typically to 0)
        if move_signals:
            sig = pd.concat(move_signals, axis=1)
            df["is_mover"] = sig.apply(
                lambda r: 1 if (r == 1).any() else (0 if (r == 0).any() else np.nan),
                axis=1
            ).astype(float)  # keep float so imputer can handle NaN
        assert isinstance(df, pd.DataFrame), "FeatureBuilder.transform must return a DataFrame"
        # age bucket
        if "age" in df.columns:
            df["age_bucket"] = pd.cut(
                df["age"].astype(float),
                bins=[0, 17, 24, 34, 44, 54, 64, 120],
                labels=["u18","18-24","25-34","35-44","45-54","55-64","65+"],
                include_lowest=True
            ).astype("category")
        
        # Weeks worked bucket
        if "weeks_worked_in_year" in df.columns:
            w = df["weeks_worked_in_year"].astype(float)
            df["weeks_bucket"] = pd.cut(
                w, bins=[-1, 0, 26, 51, 100],
                labels=["0","1-26","27-51","52"]
            ).astype("category")
        
        # Age x Education cross
        if "age_bucket" in df.columns and "education" in df.columns:
            df["age_edu"] = (df["age_bucket"].astype(str) + "|" + df["education"].astype(str)).astype("category")
        has_cols = []
        for c in ["capital_gains","capital_losses","dividends_from_stocks"]:
            if c in df.columns:
                has_cols.append((df[c] > 0).astype(int))
        if has_cols:
            df["has_any_capital"] = pd.concat(has_cols, axis=1).max(axis=1).astype(int)

        if "capital_gains" in df.columns and "capital_losses" in df.columns:
            net = (df["capital_gains"].fillna(0) - df["capital_losses"].fillna(0))
            df["cap_net"] = net
            df["log_cap_net_pos"] = np.where(net > 0, np.log1p(net), 0.0)
        # Married flag
        if "marital_stat" in df.columns:
            m = df["marital_stat"].astype(str)
            df["is_married"] = m.str.contains("Married", case=False, na=False).astype(int)

        # Union member flag
        if "member_of_a_labor_union" in df.columns:
            u = df["member_of_a_labor_union"].astype(str).str.strip().str.lower()
            df["is_union"] = u.eq("yes").astype(int)

        # Self-employed flag
        if "class_of_worker" in df.columns:
            cw = df["class_of_worker"].astype(str).str.lower()
            df["is_self_employed"] = cw.str.contains("self-employed", na=False).astype(int)
        if "migration_prev_res_in_sunbelt" in df.columns:
            s = df["migration_prev_res_in_sunbelt"].astype(str).str.strip().str.lower()
            s = s.replace({"not in universe": "missing"})
            s = s.where(s.isin(["yes","no","missing"]), "missing")
            df["prev_res_sunbelt"] = s.astype("category")
        return df
    def fit_transform(self, X, y=None, **fit_params):
        # Be explicit to avoid surprises with Mixin versions
        return self.transform(X)
    

In [4]:
# --- 2) Target Mean Encoder (renamed to avoid clash) ---
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np, pandas as pd

class TargetMeanEncoder1D(BaseEstimator, TransformerMixin):
    def __init__(self, m=50):
        self.m = m  # smoothing

    def _to_series(self, X):
        if isinstance(X, pd.Series):
            return X
        if isinstance(X, pd.DataFrame):
            if X.shape[1] != 1:
                raise ValueError("TargetMeanEncoder1D expects exactly 1 column.")
            return X.iloc[:, 0]
        arr = np.asarray(X)
        if arr.ndim == 2 and arr.shape[1] == 1:
            arr = arr[:, 0]
        return pd.Series(arr)

    def fit(self, X, y):
        s = self._to_series(X).reset_index(drop=True)
        y = pd.Series(y).reset_index(drop=True)
        self.global_mean_ = float(y.mean())
        df_ = pd.DataFrame({"cat": s, "y": y})
        means = df_.groupby("cat")["y"].mean()
        counts = df_.groupby("cat")["y"].count()
        m = self.m
        enc = (means * counts + m * self.global_mean_) / (counts + m)
        self.mapping_ = enc.to_dict()
        return self

    def transform(self, X):
        s = self._to_series(X)
        mapped = s.map(self.mapping_)
        mapped = pd.to_numeric(mapped, errors="coerce")
        out = mapped.fillna(float(self.global_mean_)).to_numpy().reshape(-1, 1)
        return out

In [5]:

# function to map various income labels to binary 0/1
def map_label_to_binary_census(s: pd.Series) -> pd.Series:
    """
    Map income label to {0,1} for Census/Adult variants.
    Handles numeric 0/1; '<=50K'/'<=50K.'; '>50K'/'50000+.'; '- 50000.'; and generic patterns.
    """
    # Case 1: already numeric 0/1
    if pd.api.types.is_numeric_dtype(s):
        uniq = set(pd.unique(s))
        if uniq <= {0, 1}:
            return s.astype(int)

    # Normalize strings: strip, lower, remove spaces and periods
    s_norm = (
        s.astype(str)
         .str.strip()
         .str.lower()
         .str.replace(r"\s+", "", regex=True)   # remove all spaces
         .str.replace(".", "", regex=False)     # remove dots
    )

    # Exact tokens commonly seen
    exact_map = {
        "<=50k": 0, "<=50k": 0, "<=50": 0, "<50k": 0, "<50": 0,
        "-50000": 0,            # maps ' - 50000.' → '-50000'
        ">50k": 1, ">50": 1,
        "50000+": 1,            # maps ' 50000+.' → '50000+'
        "50k+": 1
    }

    # If all values are in the exact_map keys
    if set(pd.unique(s_norm)) <= set(exact_map.keys()):
        return s_norm.map(exact_map).astype(int)

    # Fallback: pattern-based (covers mixed vocab)
    is_high = (
        s_norm.str.contains(r"(>50k|>50|50000\+|50k\+)", regex=True)
    )
    is_low = (
        s_norm.str.contains(r"(<\=50k|<\=50|<50k|<50|-50000)", regex=True)
    )

    y = pd.Series(np.where(is_high, 1, np.where(is_low, 0, np.nan)), index=s.index)

    # Final check
    if y.isna().any():
        print("Unmapped label examples (top 10):")
        print(s.loc[y.isna()].value_counts().head(10))
        raise ValueError("Some labels could not be mapped. Update the mapping patterns.")
    return y.astype(int)

# K-Means

In [6]:
# ===================== 1) Label mapping & build raw X =====================
TARGET_COL = "label"
y = map_label_to_binary_census(df[TARGET_COL])
print("y distribution:\n", y.value_counts(dropna=False))
assert y.nunique() == 2, "Label must contain both classes before splitting."
# --- Build sample weights from survey weight column ---
w_all = df["weight"].astype(float)
w_all = w_all / w_all.mean()  # normalize to mean=1 for numeric stability
X = df.drop(columns=[TARGET_COL])

# ===================== 2) Probe FeatureBuilder OUTPUT (no X_train needed) =====================
# We only use this to discover which columns exist AFTER FeatureBuilder.
# This does NOT use y and does NOT leak target information.
tmp = FeatureBuilder().fit_transform(X.copy())
print("FeatureBuilder output shape:", tmp.shape)


# Candidate lists we WANT to use
numeric_cols_all = [
    "age", "weeks_worked_in_year",
    "log_wage_per_hour", "log_capital_gains", "log_capital_losses", "log_dividends_from_stocks",
    "hourly_worker", "has_capital_gains", "has_capital_losses", "has_dividends_from_stocks",
    "full_year_worker", "no_work", "is_mover","has_any_capital", "cap_net", "log_cap_net_pos",
    "is_married", "is_union", "is_self_employed"
]
low_card_cats_all = [
    "sex", "marital_stat", "education", "class_of_worker",
    "citizenship", "region_of_previous_residence","age_bucket", "weeks_bucket", "age_edu",
    "prev_res_sunbelt"
]
high_card_cats_all = [
    "major_industry_code", "major_occupation_code",
    "detailed_industry_recode", "detailed_occupation_recode"
]
if "age_edu" in tmp.columns:
    nunq = tmp["age_edu"].nunique(dropna=True)
    if nunq <= 40:
        low_card_cats_all.append("age_edu")
    else:
        high_card_cats_all.append("age_edu")

# Keep only features that actually exist after FeatureBuilder
numeric_cols   = [c for c in numeric_cols_all   if c in tmp.columns]
low_card_cats  = [c for c in low_card_cats_all  if c in tmp.columns]
high_card_cats = [c for c in high_card_cats_all if c in tmp.columns]

print("numeric_cols:", numeric_cols)
print("low_card_cats:", low_card_cats)
print("high_card_cats:", high_card_cats)

# ===================== 3) Build preprocess & pipeline =====================
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# OneHot compatibility across sklearn versions
try:
    onehot = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    onehot = OneHotEncoder(handle_unknown="ignore", sparse=False)

num_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

# Target-mean encoder blocks for high-card categoricals
tgt_blocks = [(f"tgt_{col}", TargetMeanEncoder1D(m=50), [col]) for col in high_card_cats]

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("ohe", onehot, low_card_cats),
        *tgt_blocks
    ],
    remainder="drop"
)


y distribution:
 label
0    183912
1     12382
Name: count, dtype: int64
FeatureBuilder output shape: (196294, 56)
numeric_cols: ['age', 'weeks_worked_in_year', 'log_wage_per_hour', 'log_capital_gains', 'log_capital_losses', 'log_dividends_from_stocks', 'hourly_worker', 'has_capital_gains', 'has_capital_losses', 'has_dividends_from_stocks', 'full_year_worker', 'no_work', 'is_mover', 'has_any_capital', 'cap_net', 'log_cap_net_pos', 'is_married', 'is_union', 'is_self_employed']
low_card_cats: ['sex', 'marital_stat', 'education', 'class_of_worker', 'citizenship', 'region_of_previous_residence', 'age_bucket', 'weeks_bucket', 'age_edu', 'prev_res_sunbelt']
high_card_cats: ['major_industry_code', 'major_occupation_code', 'detailed_industry_recode', 'detailed_occupation_recode', 'age_edu']


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42
)
print("Shapes:", X_train.shape, X_val.shape, X_test.shape)


Shapes: (117776, 41) (39259, 41) (39259, 41)


In [23]:
w_trainval = w_all.loc[X_trainval.index]
w_train    = w_all.loc[X_train.index]
w_val      = w_all.loc[X_val.index]
w_test     = w_all.loc[X_test.index]

In [12]:
# ==== Build sample weights from survey 'weight' (after you have X_trainval / y_trainval) ====
w_all = df["weight"].astype(float)
w_all = w_all / w_all.mean()  # normalize for numeric stability

w_trainval = w_all.loc[X_trainval.index]

# (optional) compute weighted class ratio for tree models
neg_w = w_trainval[y_trainval == 0].sum()
pos_w = w_trainval[y_trainval == 1].sum()
spw_weighted = float(neg_w / max(pos_w, 1e-12))

In [ ]:


# ---------- 1) Unsupervised-friendly encoder for high-card categoricals ----------
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import FeatureHasher

class HashingCats(BaseEstimator, TransformerMixin):
    """Hashing trick for multiple categorical columns: fast and target-free."""
    def __init__(self, cols, n_features=2**12, alternate_sign=False):
        self.cols = cols
        self.n_features = n_features
        self.alternate_sign = alternate_sign
        self.hasher = FeatureHasher(
            n_features=n_features,
            input_type="string",
            alternate_sign=alternate_sign
        )

    def fit(self, X, y=None):
        return self  # stateless

    def transform(self, X):
        if not self.cols:
            # return empty sparse matrix with right #rows
            from scipy import sparse
            return sparse.csr_matrix((len(X), self.n_features))
        # build tokens like "col=value" per row
        tokens = (
            X[self.cols].astype(str)
              .apply(lambda r: [f"{c}={r[c]}" for c in self.cols], axis=1)
              .tolist()
        )
        return self.hasher.transform(tokens)

# ---------- 2) Build unsupervised preprocess (no target-mean encoding) ----------
# OneHot (version-compat)
try:
    onehot = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    onehot = OneHotEncoder(handle_unknown="ignore", sparse=True)

num_pipe_unsup = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True, with_std=True))
])

hash_block = ("hash", HashingCats(cols=[], n_features=2**12), None)  # placeholder; set cols below

# Extend low-card with engineered buckets if present

low_card_all = [c for c in low_card_cats  if c in tmp.columns]
high_card_all = [c for c in high_card_cats if c in tmp.columns]

# update hashing block with actual high-card cols
hash_block = ("hash", HashingCats(cols=high_card_all, n_features=2**12), high_card_all)

pre_unsup = ColumnTransformer(
    transformers=[
        ("num",  num_pipe_unsup, numeric_cols),
        ("ohe",  onehot, low_card_all),
        hash_block,
    ],
    remainder="drop",              # keep only what we specified
    sparse_threshold=1.0           # keep it sparse so SVD can handle efficiently
)

# Full feature builder + preprocess + SVD pipeline
prep_pipe = Pipeline([
    ("feat", FeatureBuilder()),
    ("prep", pre_unsup),
    ("svd", TruncatedSVD(n_components=30, random_state=42))  # 20~50 is often plenty
])

# ---------- 3) Prepare weights (normalized) ----------
w_all = df["weight"].astype(float)
w_all = w_all / w_all.mean()

# Split you already have: X_trainval, X_test, etc.
# Fit prep on TrainVal to avoid any look-ahead
Z_trainval = prep_pipe.fit_transform(X_trainval)
Z_test     = prep_pipe.transform(X_test)

# ---------- 4) Pick K by a quick silhouette sweep (unweighted) ----------
k_candidates = list(range(4, 9))
sil_scores = {}
for k in k_candidates:
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    km.fit(Z_trainval, sample_weight=w_trainval)  # weighted clustering
    labels = km.labels_
    # silhouette_score has no sample_weight; use as heuristic
    try:
        sil = silhouette_score(Z_trainval, labels, metric="euclidean")
    except Exception:
        sil = np.nan
    sil_scores[k] = sil
best_k = max(sil_scores, key=lambda kk: (sil_scores[kk] if sil_scores[kk] == sil_scores[kk] else -1))
print("Silhouette by k:", sil_scores, " -> best_k:", best_k)

# ---------- 5) Fit final model with best_k on TrainVal ----------
kmeans = KMeans(n_clusters=best_k, n_init="auto", random_state=42)
kmeans.fit(Z_trainval, sample_weight=w_trainval)
labels_trainval = pd.Series(kmeans.labels_, index=X_trainval.index, name="segment")

# also assign segments on Test
labels_test = pd.Series(kmeans.predict(Z_test), index=X_test.index, name="segment")

# ---------- 6) Build weighted profiles per segment ----------
# join back with original (post-FeatureBuilder) features for readable profiling
Xtv_feat = prep_pipe.named_steps["feat"].transform(X_trainval.copy())  # DataFrame with engineered columns
Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

def wmean_by_seg(s_values, seg, w):
    num = (s_values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(s_values.name)

def wprop_table(cat_series, seg, w):
    # returns segment x category proportions (weighted)
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

# pick a few numeric & categorical for marketing-readable profiles
num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                           "full_year_worker","no_work"] if c in Xtv_feat.columns]
cat_to_show = [c for c in ["sex","education","class_of_worker","citizenship",
                           "region_of_previous_residence","age_bucket","weeks_bucket",
                           "is_union","is_self_employed","has_any_capital"] if c in Xtv_feat.columns]

w_seg = w_trainval.copy()
seg_trainval = labels_trainval.reindex(X_trainval.index)

# numeric summaries
num_profiles = []
for c in num_to_show:
    num_profiles.append(wmean_by_seg(Xtv_feat[c], seg_trainval, w_seg))
num_profiles = pd.concat(num_profiles, axis=1)
print("\n[Numeric profile | weighted means]")
print(num_profiles.round(3))

# categorical summaries: show top-3 categories per segment for each field
print("\n[Categorical profile | top categories per segment]")
for c in cat_to_show:
    tab = wprop_table(Xtv_feat[c], seg_trainval, w_seg)
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((top3*100).round(1))

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Silhouette by k: {4: 0.23369178680130037, 5: 0.24590672378290468, 6: 0.2823499812513904, 7: 0.2585143891999381, 8: 0.32326370240677066}  -> best_k: 8


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[Numeric profile | weighted means]
            age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
0        58.336                 1.459              0.000     0.059             0.000    0.911
1        37.271                45.928              0.000     0.102             0.733    0.000
2        42.448                48.330              5.873     0.062             0.798    0.002
3        35.985                44.844              6.722     0.089             0.706    0.028
4        48.825                38.565              0.005     0.067             0.662    0.198
5        43.866                41.381              0.435     0.076             0.698    0.118
6        50.025                32.815              0.004     0.049             0.549    0.297
7         9.494                 0.521              0.000     0.089             0.000    0.958

[Categorical profile | 

In [19]:
# === KMeans K-selection with multiple indices, repeats, and weighted-share constraint ===
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)

def _safe_kmeans(k, seed):
    """Return a KMeans instance compatible with different sklearn versions."""
    try:
        return KMeans(n_clusters=k, n_init="auto", random_state=seed)
    except TypeError:
        return KMeans(n_clusters=k, n_init=10, random_state=seed)

def _weighted_cluster_share(labels, weights):
    """Compute weighted share per cluster; returns a pd.Series indexed by label."""
    weights = np.asarray(weights).reshape(-1)
    df = pd.DataFrame({"lab": labels, "w": weights})
    return df.groupby("lab")["w"].sum() / df["w"].sum()

def pick_kmeans_k(
    Z,
    weights=None,
    k_list=range(4, 11),
    n_repeats=8,
    min_weighted_share=0.02,   # drop runs that create tiny weighted clusters
    random_state=42,
    verbose=True,
):
    """
    Evaluate KMeans over k_list with multiple random seeds.
    Primary rank: mean silhouette (higher is better).
    Tie-breakers: mean CH (higher), mean DB (lower), mean inertia (lower).
    Enforces a minimum weighted cluster share per run.
    Returns: summary_df, best_k, final_model
    """
    Z = np.asarray(Z)  # ensure array-like
    n = Z.shape[0]
    if weights is None:
        weights = np.ones(n, float)
    weights = np.asarray(weights).reshape(-1)

    rows = []      # per-run records
    agg = []       # per-k aggregates

    # ---- sweep K and repeats ----
    for k in k_list:
        for rep in range(n_repeats):
            seed = random_state + rep
            km = _safe_kmeans(k, seed)
            # fit with sample weights (sklearn KMeans supports sample_weight)
            km.fit(Z, sample_weight=weights)
            labels = km.labels_

            # filter out degenerate runs with tiny weighted clusters
            share = _weighted_cluster_share(labels, weights)
            min_share = share.min()

            # compute metrics (silhouette/DB/CH ignore weights by sklearn design)
            try:
                sil = silhouette_score(Z, labels, metric="euclidean")
            except Exception:
                sil = np.nan
            try:
                db = davies_bouldin_score(Z, labels)
            except Exception:
                db = np.nan
            try:
                ch = calinski_harabasz_score(Z, labels)
            except Exception:
                ch = np.nan

            rows.append({
                "k": k,
                "rep": rep,
                "seed": seed,
                "silhouette": sil,
                "db": db,
                "ch": ch,
                "inertia": float(km.inertia_),
                "min_weighted_share": float(min_share),
                "valid": bool(min_share >= min_weighted_share),
            })

    runs = pd.DataFrame(rows)

    # ---- aggregate per k only over valid runs ----
    def _agg(df):
        return pd.Series({
            "n_valid": df["valid"].sum(),
            "n_total": len(df),
            "sil_mean": df.loc[df["valid"], "silhouette"].mean(),
            "sil_std":  df.loc[df["valid"], "silhouette"].std(),
            "ch_mean":  df.loc[df["valid"], "ch"].mean(),
            "db_mean":  df.loc[df["valid"], "db"].mean(),
            "inertia_mean": df.loc[df["valid"], "inertia"].mean(),
            "min_share_min": df.loc[df["valid"], "min_weighted_share"].min(),
            "min_share_mean": df.loc[df["valid"], "min_weighted_share"].mean(),
        })

    summary = runs.groupby("k").apply(_agg).sort_index()

    # ---- pick best_k by a robust composite rule ----
    # Primary: highest sil_mean (nan -> -inf). Tie-breakers: higher ch_mean, lower db_mean, lower inertia_mean.
    def rank_key(krow):
        s = krow["sil_mean"]
        ch = krow["ch_mean"]
        db = krow["db_mean"]
        ine = krow["inertia_mean"]
        # convert NaNs to conservative values
        s  = -np.inf if pd.isna(s)  else s
        ch = -np.inf if pd.isna(ch) else ch
        db =  np.inf if pd.isna(db) else db
        ine=  np.inf if pd.isna(ine) else ine
        # Higher sil, higher CH, lower DB, lower inertia
        return (-s, -ch, db, ine)

    # drop k with zero valid runs
    summary_valid = summary[summary["n_valid"] > 0].copy()
    if summary_valid.empty:
        raise RuntimeError("All runs invalid under the min_weighted_share constraint. "
                           "Relax min_weighted_share or check data preprocessing.")

    best_k = min(summary_valid.index, key=lambda kk: rank_key(summary_valid.loc[kk]))

    # ---- fit final model on all data (best_k) with a fresh seed ----
    final_seed = random_state + 999
    final_km = _safe_kmeans(best_k, final_seed)
    final_km.fit(Z, sample_weight=weights)

    if verbose:
        print("\n[KMeans CV-like sweep] summary (valid runs only):")
        print(summary_valid[[
            "n_valid","n_total","sil_mean","sil_std","ch_mean","db_mean","inertia_mean",
            "min_share_min","min_share_mean"
        ]].round(3).sort_values("sil_mean", ascending=False))
        print(f"\n[Selected best_k] {best_k}  |  "
              f"sil_mean={summary_valid.loc[best_k,'sil_mean']:.3f}  |  "
              f"CH={summary_valid.loc[best_k,'ch_mean']:.1f}  |  "
              f"DB={summary_valid.loc[best_k,'db_mean']:.3f}  |  "
              f"min_weighted_share≈{summary_valid.loc[best_k,'min_share_min']:.3f}")

    return summary_valid, int(best_k), final_km

In [20]:

summary, best_k, kmeans_final = pick_kmeans_k(
    Z=Z_trainval,
    weights=w_trainval.values,
    k_list=range(4, 11),      
    n_repeats=8,              
    min_weighted_share=0.02,  
    random_state=42,
    verbose=True
)


labels_trainval = pd.Series(kmeans_final.labels_, index=X_trainval.index, name="segment")
labels_test = pd.Series(kmeans_final.predict(Z_test), index=X_test.index, name="segment")

wshare = 100.0 * w_trainval.groupby(labels_trainval).sum() / w_trainval.sum()
print("\n[Final KMeans] weighted_share (%)\n", wshare.round(1))


[KMeans CV-like sweep] summary (valid runs only):
    n_valid  n_total  sil_mean  sil_std    ch_mean  db_mean  inertia_mean  min_share_min  min_share_mean
k                                                                                                       
10      1.0      8.0     0.339      NaN  32314.869    1.316   1403200.261          0.021           0.021
9       3.0      8.0     0.328    0.002  34643.118    1.290   1450862.010          0.021           0.021
8       6.0      8.0     0.327    0.019  34860.245    1.288   1572844.411          0.021           0.021
7       5.0      8.0     0.293    0.023  29923.507    1.413   1878169.348          0.021           0.027
6       6.0      8.0     0.282    0.019  27206.178    1.468   2151877.292          0.021           0.029
4       8.0      8.0     0.264    0.035  28258.186    1.540   2614253.112          0.021           0.042
5       8.0      8.0     0.256    0.027  26760.512    1.562   2393291.999          0.021           0.031

[Se

/var/folders/rt/qjkw9ljx5h3fr3tbpb26t3s00000gn/T/ipykernel_73189/2214396932.py:104: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = runs.groupby("k").apply(_agg).sort_index()


In [ ]:
# === Print final KMeans segmentation results ===


# Expect these to exist from your run:
# - kmeans_final  (fitted on Z_trainval with best_k)
# - Z_test, Z_trainval
# - labels_trainval, labels_test  (we created them when picking K)
# - prep_pipe  (FeatureBuilder + preprocess + SVD pipeline used for clustering)
# - X_trainval, X_test, y_trainval, y_test (y_* optional for supervised lift)
# - w_trainval, w_test  (survey weights)

# 1) Weighted share by segment (TrainVal & Test)
wshare_tv = 100.0 * w_trainval.groupby(labels_trainval).sum() / w_trainval.sum()
wshare_te = 100.0 * w_test.groupby(labels_test).sum() / w_test.sum()
print("\n[Weighted share % | TrainVal]\n", wshare_tv.round(1).sort_values(ascending=False))
print("\n[Weighted share % | Test]\n", wshare_te.round(1).sort_values(ascending=False))

# 2) Build a readable feature frame (post-FeatureBuilder) for profiling
Xtv_feat = prep_pipe.named_steps["feat"].transform(X_trainval.copy())  # DataFrame with engineered cols
Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

# ---- helpers (weighted means / weighted category table) ----
def wmean_by_seg(s_values, seg, w):
    num = (s_values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(s_values.name)

def wprop_table(cat_series, seg, w):
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

# 3) Numeric profile (weighted means)
num_to_show = [c for c in [
    "age","weeks_worked_in_year","log_wage_per_hour",
    "is_mover","full_year_worker","no_work"
] if c in Xtv_feat.columns]

if num_to_show:
    num_profiles = pd.concat(
        [wmean_by_seg(Xtv_feat[c], labels_trainval, w_trainval) for c in num_to_show],
        axis=1
    ).round(3)
    print("\n[Numeric profile | weighted means]\n", num_profiles)
else:
    print("\n[Numeric profile] No numeric columns found in Xtv_feat subset.")

# 4) Categorical profile (top-3 weighted % per segment)
cat_to_show = [c for c in [
    "sex","education","class_of_worker","citizenship",
    "region_of_previous_residence","age_bucket","weeks_bucket",
    "is_union","is_self_employed","has_any_capital"
] if c in Xtv_feat.columns]

print("\n[Categorical profile | top categories per segment]")
for c in cat_to_show:
    tab = wprop_table(Xtv_feat[c], labels_trainval, w_trainval)  # segment x categories (proportions)
    # For each segment (row), take top-3 categories
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((top3 * 100).round(1))

# 5) (Optional) Supervised lift by segment (weighted positive rate), if y_* available
try:
    lift_tv = wmean_by_seg(y_trainval.astype(int), labels_trainval, w_trainval).rename("pos_rate")
    lift_te = wmean_by_seg(y_test.astype(int), labels_test, w_test).rename("pos_rate")
    print("\n[Supervised lift | weighted positive rate by segment]")
    print("TrainVal:\n", (100*lift_tv).round(1))
    print("Test:\n", (100*lift_te).round(1))
except Exception:
    pass  # y_* not present; skip

# 6) A compact marketing summary table (one line per segment)
#    You can customize the fields below to match your storytelling.
def safe_pick_prop(tab, cat, default=np.nan):
    """Pick weighted proportion of a category from a segment x category table."""
    return tab[cat] if (cat in tab.columns) else pd.Series(default, index=tab.index)

summary = pd.DataFrame(index=sorted(labels_trainval.unique()))
summary["share_%"] = wshare_tv.reindex(summary.index).round(1)

# add a few numeric KPIs
for col in ["age","weeks_worked_in_year","log_wage_per_hour"]:
    if col in Xtv_feat.columns:
        summary[col] = wmean_by_seg(Xtv_feat[col], labels_trainval, w_trainval).reindex(summary.index)

# add a few categorical KPIs (as %)
if "sex" in Xtv_feat.columns:
    sex_tab = wprop_table(Xtv_feat["sex"], labels_trainval, w_trainval)
    summary["female_%"] = (100 * safe_pick_prop(sex_tab, "Female", 0)).reindex(summary.index)

if "education" in Xtv_feat.columns:
    edu_tab = wprop_table(Xtv_feat["education"], labels_trainval, w_trainval)
    summary["bachelor_%"] = (100 * safe_pick_prop(edu_tab, "Bachelors degree(BA AB BS)", 0)).reindex(summary.index)

if "class_of_worker" in Xtv_feat.columns:
    cow_tab = wprop_table(Xtv_feat["class_of_worker"], labels_trainval, w_trainval)
    summary["private_worker_%"] = (100 * safe_pick_prop(cow_tab, "Private", 0)).reindex(summary.index)

# optional supervised rate (if y available)
try:
    summary["pos_rate_%"] = (100 * lift_tv).reindex(summary.index)
except Exception:
    pass

print("\n[Marketing summary | TrainVal]")
print(summary.round(2).sort_values("share_%", ascending=False))

# 7) (Optional) Save segment assignment for downstream usage
seg_trainval_df = pd.DataFrame({
    "segment": labels_trainval,
    "weight": w_trainval.values,
}, index=X_trainval.index)

# if y is present, include it for later lift/targeting analyses
try:
    seg_trainval_df["y"] = y_trainval.values
except Exception:
    pass

# Example save (uncomment if you want a CSV)
# seg_trainval_df.to_csv("segments_trainval.csv", index=True)


[Weighted share % | TrainVal]
 segment
0    25.4
2    23.4
3    16.0
4     8.6
9     6.8
5     5.6
1     4.3
6     4.2
8     3.8
7     2.1
Name: weight, dtype: float64

[Weighted share % | Test]
 segment
0    25.1
2    23.5
3    15.8
4     8.8
9     6.8
5     5.5
1     4.4
6     4.3
8     3.8
7     2.0
Name: weight, dtype: float64


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[Numeric profile | weighted means]
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
0         9.498                 0.519              0.000     0.000             0.000    0.958
1        43.815                44.629              0.000     0.001             0.716    0.029
2        35.964                45.861              0.000     0.000             0.732    0.000
3        59.791                 1.017              0.000     0.000             0.000    0.930
4        50.082                32.671              0.004     0.048             0.547    0.300
5        35.985                44.844              6.722     0.089             0.706    0.028
6        43.511                45.051              0.000     0.061             0.743    0.045
7        43.866                41.381              0.435     0.076             0.698    0.118
8        48.341        

# HDBSCAN

In [ ]:
# --- 1) Build embeddings (dense) ---
Z_trainval = prep_pipe.fit_transform(X_trainval)
Z_test     = prep_pipe.transform(X_test)

# --- 2) Weighted PPS subsample for fitting HDBSCAN ---
import hdbscan
rng = np.random.default_rng(42)
n_sub = min(len(Z_trainval), 60000)  # cap for speed
p = (w_trainval.values / w_trainval.values.sum())
idx_sub = rng.choice(len(Z_trainval), size=n_sub, replace=False, p=p)
Z_sub = Z_trainval[idx_sub]

# Set min_cluster_size relative to the subsample size you actually fit on
min_cluster_size = max(200, int(0.01 * n_sub))

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=min_cluster_size,
    min_samples=None,
    metric="euclidean",
    cluster_selection_method="eom",
    approx_min_span_tree=True,
    prediction_data=True   # must be True for approximate_predict
)

# Fit on the subsample (no sample_weight accepted by HDBSCAN.fit in stable releases)
clusterer.fit(Z_sub)

# --- 3) Assign labels for the FULL TrainVal via approximate_predict ---
# Do NOT use clusterer.labels_ here. It only has length n_sub.
tr_labels, tr_strength = hdbscan.approximate_predict(clusterer, Z_trainval)
labels_trainval = pd.Series(tr_labels, index=X_trainval.index, name="segment")

# Assign for Test as well
te_labels, te_strength = hdbscan.approximate_predict(clusterer, Z_test)
labels_test = pd.Series(te_labels, index=X_test.index, name="segment")

# --- 4) Weighted summaries (use survey weights only in profiling/evaluation) ---
# weighted share (including noise = -1)
wshare = 100.0 * w_trainval.groupby(labels_trainval).sum() / w_trainval.sum()
print("\n[HDBSCAN PPS] weighted_share (%)\n", wshare.round(1))

# If you want profiles:
Xtv_feat = prep_pipe.named_steps["feat"].transform(X_trainval.copy())

def wmean_by_seg(s_values, seg, w):
    num = (s_values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(s_values.name)

def wprop_table(cat_series, seg, w):
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                           "full_year_worker","no_work"] if c in Xtv_feat.columns]
num_prof = pd.concat([wmean_by_seg(Xtv_feat[c], labels_trainval, w_trainval) for c in num_to_show], axis=1)
print("\n[HDBSCAN] Numeric profile (weighted means)\n", num_prof.round(3))

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



[HDBSCAN PPS] weighted_share (%)
 segment
-1     25.0
 0      2.0
 1      3.4
 2      1.1
 3      4.1
 4      6.9
 5      3.5
 6      1.2
 7      2.5
 8      2.6
 9      4.8
 10    16.4
 11     9.7
 12    10.2
 13     1.5
 14     1.9
 15     1.4
 16     1.8
Name: weight, dtype: float64


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[HDBSCAN] Numeric profile (weighted means)
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
-1       39.957                22.009              0.239     0.160             0.210    0.371
 0       43.856                41.237              0.341     0.075             0.696    0.120
 1       48.546                38.486              0.277     0.066             0.658    0.198
 2       40.926                49.008              4.871     0.015             0.833    0.007
 3       34.523                44.425              6.629     0.063             0.692    0.028
 4       50.862                30.385              0.000     0.000             0.518    0.354
 5       43.493                47.654              0.000     0.000             0.801    0.000
 6       29.984                51.505              0.000     1.000             0.974    0.000
 7       11.211

In [25]:
# --- 1) Build embeddings (dense) ---
Z_trainval = prep_pipe.fit_transform(X_trainval)
Z_test     = prep_pipe.transform(X_test)

# --- 2) Weighted PPS subsample for fitting HDBSCAN ---
import hdbscan
rng = np.random.default_rng(42)
n_sub = min(len(Z_trainval), 60000)  # cap for speed
p = (w_trainval.values / w_trainval.values.sum())
idx_sub = rng.choice(len(Z_trainval), size=n_sub, replace=False, p=p)
Z_sub = Z_trainval[idx_sub]

# Set min_cluster_size relative to the subsample size you actually fit on
min_cluster_size = max(200, int(0.01 * n_sub))

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=min_cluster_size,
    min_samples=None,
    metric="euclidean",
    cluster_selection_method="eom",
    approx_min_span_tree=True,
    prediction_data=True   # must be True for approximate_predict
)

# Fit on the subsample (no sample_weight accepted by HDBSCAN.fit in stable releases)
clusterer.fit(Z_sub)

# --- 3) Assign labels for the FULL TrainVal via approximate_predict ---
# Do NOT use clusterer.labels_ here. It only has length n_sub.
tr_labels, tr_strength = hdbscan.approximate_predict(clusterer, Z_trainval)
labels_trainval = pd.Series(tr_labels, index=X_trainval.index, name="segment")

# Assign for Test as well
te_labels, te_strength = hdbscan.approximate_predict(clusterer, Z_test)
labels_test = pd.Series(te_labels, index=X_test.index, name="segment")

# --- 4) Weighted summaries (use survey weights only in profiling/evaluation) ---
# weighted share (including noise = -1)
wshare = 100.0 * w_trainval.groupby(labels_trainval).sum() / w_trainval.sum()
print("\n[HDBSCAN PPS] weighted_share (%)\n", wshare.round(1))

# If you want profiles:
Xtv_feat = prep_pipe.named_steps["feat"].transform(X_trainval.copy())

def wmean_by_seg(s_values, seg, w):
    num = (s_values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(s_values.name)

def wprop_table(cat_series, seg, w):
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                           "full_year_worker","no_work"] if c in Xtv_feat.columns]
num_prof = pd.concat([wmean_by_seg(Xtv_feat[c], labels_trainval, w_trainval) for c in num_to_show], axis=1)
print("\n[HDBSCAN] Numeric profile (weighted means)\n", num_prof.round(3))
# After: clusterer.fit(...) and Z_test = prep_pipe.transform(X_test)


# assign test labels
te_labels, te_strength = hdbscan.approximate_predict(clusterer, Z_test)
labels_test = pd.Series(te_labels, index=X_test.index, name="segment")

# weighted share on TEST (includes noise = -1)
wshare_test = 100.0 * w_test.groupby(labels_test).sum() / w_test.sum()
print("\n[HDBSCAN] weighted_share TEST (%)\n", wshare_test.round(1))

# optional: test silhouette (internal quality; unweighted)
from sklearn.metrics import silhouette_score
print("[HDBSCAN] silhouette_test:", silhouette_score(Z_test, labels_test))

# optional: supervised lift by segment on TEST (if y_test exists)
lift_te = ((w_test * y_test).groupby(labels_test).sum() /
           w_test.groupby(labels_test).sum())
print("[HDBSCAN] pos_rate by segment (TEST, weighted %):\n", (100*lift_te).round(1))

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



[HDBSCAN PPS] weighted_share (%)
 segment
-1     25.0
 0      2.0
 1      3.4
 2      1.1
 3      4.1
 4      6.9
 5      3.5
 6      1.2
 7      2.5
 8      2.6
 9      4.8
 10    16.4
 11     9.7
 12    10.2
 13     1.5
 14     1.9
 15     1.4
 16     1.8
Name: weight, dtype: float64


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[HDBSCAN] Numeric profile (weighted means)
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
-1       39.957                22.009              0.239     0.160             0.210    0.371
 0       43.856                41.237              0.341     0.075             0.696    0.120
 1       48.546                38.486              0.277     0.066             0.658    0.198
 2       40.926                49.008              4.871     0.015             0.833    0.007
 3       34.523                44.425              6.629     0.063             0.692    0.028
 4       50.862                30.385              0.000     0.000             0.518    0.354
 5       43.493                47.654              0.000     0.000             0.801    0.000
 6       29.984                51.505              0.000     1.000             0.974    0.000
 7       11.211

In [26]:
# --- 0) Ensure we have engineered features for TEST (post-FeatureBuilder) ---
# If you already have Xte_feat from earlier, you can skip this block.
Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

# --- 1) Helpers (weighted mean & weighted categorical % by segment) ---
def wmean_by_seg(values, seg, w):
    """Weighted mean of a numeric Series grouped by segment."""
    num = (values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(values.name)

def wprop_table(cat_series, seg, w):
    """Weighted proportion table (segment x category)."""
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

# --- 2) Basic segment sizes and strength on TEST ---
# (labels_test and te_strength come from hdbscan.approximate_predict)
seg_te = labels_test.reindex(X_test.index)
w_te = w_test.reindex(X_test.index)

weighted_share_te = 100.0 * w_te.groupby(seg_te).sum() / w_te.sum()
print("\n[HDBSCAN] weighted_share TEST (%)\n", weighted_share_te.round(1))

# Optional: average membership strength per segment (higher ~ more confident)
strength_te = pd.Series(te_strength, index=X_test.index, name="strength")
avg_strength = wmean_by_seg(strength_te, seg_te, w_te)
print("\n[HDBSCAN] avg membership strength by segment (TEST)\n", avg_strength.round(3))

# --- 3) Numeric profile (weighted means) on TEST ---
num_to_show = [
    c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                "full_year_worker","no_work"] if c in Xte_feat.columns
]
num_profiles_test = pd.concat([wmean_by_seg(Xte_feat[c], seg_te, w_te) for c in num_to_show], axis=1)
print("\n[HDBSCAN] Numeric profile (weighted means) — TEST\n", num_profiles_test.round(3))

# --- 4) Categorical profile (top-3 per segment) on TEST ---
cat_to_show = [
    c for c in ["sex","education","class_of_worker","citizenship",
                "region_of_previous_residence","age_bucket","weeks_bucket",
                "is_union","is_self_employed","has_any_capital"]
    if c in Xte_feat.columns
]

print("\n[HDBSCAN] Categorical profile (top-3 per segment) — TEST")
for c in cat_to_show:
    tab = wprop_table(Xte_feat[c], seg_te, w_te)  # segment x category proportions
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((top3 * 100).round(1))

# --- 5) Supervised lift on TEST (if you have y_test): pos rate and lift vs overall ---
if 'y_test' in globals():
    pos_rate_seg = ((w_te * y_test).groupby(seg_te).sum() / w_te.groupby(seg_te).sum())
    overall_pos = float((w_te * y_test).sum() / w_te.sum())
    lift = (pos_rate_seg / overall_pos).rename("lift")
    pr_table = pd.DataFrame({
        "weighted_share_%": weighted_share_te.round(1),
        "pos_rate_%": (100 * pos_rate_seg).round(1),
        "lift": lift.round(2),
        "avg_strength": avg_strength.round(3),
    }).sort_values("lift", ascending=False)
    print("\n[HDBSCAN] Segment performance on TEST (weighted)\n", pr_table)


[HDBSCAN] weighted_share TEST (%)
 segment
-1     25.2
 0      2.0
 1      3.4
 2      1.1
 3      4.1
 4      7.1
 5      3.6
 6      1.2
 7      2.5
 8      2.6
 9      4.8
 10    16.3
 11     9.5
 12    10.2
 13     1.3
 14     1.8
 15     1.5
 16     1.8
Name: weight, dtype: float64

[HDBSCAN] avg membership strength by segment (TEST)
 segment
-1     0.000
 0     0.916
 1     0.849
 2     0.999
 3     0.862
 4     0.966
 5     0.888
 6     0.997
 7     0.891
 8     0.963
 9     0.589
 10    0.823
 11    0.958
 12    0.960
 13    0.825
 14    0.933
 15    0.835
 16    0.992
Name: strength, dtype: float64

[HDBSCAN] Numeric profile (weighted means) — TEST
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
-1       39.770                22.012              0.230     0.157             0.209    0.372
 0       43.456                42.037             

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# === HDBSCAN quick tuning on TRAINVAL (unsupervised only), then final TEST report ===


# 0) Embeddings: fit prep on trainval, transform both (no leakage into test)
Z_tv  = prep_pipe.fit_transform(X_trainval)
Z_te  = prep_pipe.transform(X_test)

# 1) PPS subsample on trainval for fitting speed (HDBSCAN.fit has no sample_weight)
rng      = np.random.default_rng(42)
N_CAP    = 60000
n_tv     = len(Z_tv)
n_sub    = int(min(n_tv, N_CAP))
p_tv     = w_trainval.values / w_trainval.values.sum()
idx_sub  = rng.choice(n_tv, size=n_sub, replace=False, p=p_tv)
Z_sub    = Z_tv[idx_sub]

def weighted_share_by_label(labels, w):
    w = pd.Series(w)
    lab = pd.Series(labels, index=w.index[:len(labels)])
    return w.groupby(lab).sum() / w.sum()

def min_weighted_cluster_share(labels, w, ignore_noise=True):
    lab = pd.Series(labels)
    if ignore_noise:
        mask = (lab != -1)
        lab = lab[mask]
        w   = pd.Series(w).loc[mask.values]
    sh = weighted_share_by_label(lab.values, w.values)
    return float(sh.min()) if len(sh) else 0.0

# 2) Tiny grid (fast)
min_cluster_size_fracs = [0.005, 0.01, 0.02]   # ~0.5%-2% of subsample
min_samples_list       = [None, 1, 5]
methods                = ["eom", "leaf"]

MIN_CLUSTER_SHARE = 0.01   # drop configs that create tiny clusters (<1% weighted)
NOISE_TARGET      = 0.20   # prefer ~20% noise; soft preference
NOISE_TOL         = 0.20

rows = []
for frac in min_cluster_size_fracs:
    mcs = max(100, int(frac * n_sub))
    for ms in min_samples_list:
        for meth in methods:
            clus = hdbscan.HDBSCAN(
                min_cluster_size=mcs,
                min_samples=ms,
                metric="euclidean",
                cluster_selection_method=meth,
                approx_min_span_tree=True,
                prediction_data=True
            )
            # fit on subsample
            clus.fit(Z_sub)
            # assign all trainval
            tv_labels, _ = hdbscan.approximate_predict(clus, Z_tv)

            # metrics on trainval (internal-only)
            try:
                sil = silhouette_score(Z_tv, tv_labels) if len(np.unique(tv_labels)) > 1 else np.nan
            except Exception:
                sil = np.nan
            share_tv  = weighted_share_by_label(tv_labels, w_trainval.values)
            noise_tv  = float(share_tv.get(-1, 0.0))
            min_share = min_weighted_cluster_share(tv_labels, w_trainval.values, ignore_noise=True)
            valid     = (min_share >= MIN_CLUSTER_SHARE) and (np.unique(tv_labels[tv_labels!=-1]).size >= 2)

            # rank: ↑sil, noise close to target, no tiny clusters
            noise_dev = abs(noise_tv - NOISE_TARGET)
            score = (0.8 * np.nan_to_num(sil, nan=-1.0) - 0.2 * (noise_dev / NOISE_TOL))

            rows.append({
                "mcs": mcs, "min_samples": ms, "method": meth,
                "sil_tv": sil, "noise_tv": noise_tv, "min_share_tv": min_share,
                "valid": valid, "score": score
            })

tune = pd.DataFrame(rows)
tune_v = tune[tune.valid].copy()
if tune_v.empty:
    raise RuntimeError("All HDBSCAN configs invalid under constraints; relax MIN_CLUSTER_SHARE or increase min_cluster_size.")
tune_v = tune_v.sort_values(["score","sil_tv"], ascending=[False, False])

print("\n[HDBSCAN|trainval tuning] top-5")
print(tune_v.head(5).round(3))

best = tune_v.iloc[0].to_dict()
best_params = {
    "min_cluster_size": int(best["mcs"]),
    "min_samples": None if pd.isna(best["min_samples"]) else int(best["min_samples"]),
    "cluster_selection_method": best["method"]
}
print("\n[Chosen params]", best_params)

# 3) Refit on trainval with chosen params (subsample again for speed), then report on TEST
idx_sub2 = rng.choice(n_tv, size=n_sub, replace=False, p=p_tv)
clus_final = hdbscan.HDBSCAN(
    min_cluster_size=best_params["min_cluster_size"],
    min_samples=best_params["min_samples"],
    metric="euclidean",
    cluster_selection_method=best_params["cluster_selection_method"],
    approx_min_span_tree=True,
    prediction_data=True
).fit(Z_tv[idx_sub2])

# assign TEST
te_labels, te_strength = hdbscan.approximate_predict(clus_final, Z_te)
labels_test = pd.Series(te_labels, index=X_test.index, name="segment")
strength_te = pd.Series(te_strength, index=X_test.index, name="strength")

# 4) TEST report 
w_te = w_test.reindex(X_test.index)
weighted_share_te = 100.0 * w_te.groupby(labels_test).sum() / w_te.sum()
print("\n[HDBSCAN] weighted_share TEST (%)\n", weighted_share_te.round(1))

def wmean_by_seg(values, seg, w):
    num = (values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(values.name)

avg_strength = wmean_by_seg(strength_te, labels_test, w_te)
print("\n[HDBSCAN] avg membership strength by segment (TEST)\n", avg_strength.round(3))

Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

# numeric profile
num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                           "full_year_worker","no_work"] if c in Xte_feat.columns]
num_prof_te = pd.concat([wmean_by_seg(Xte_feat[c], labels_test, w_te) for c in num_to_show], axis=1)
print("\n[HDBSCAN] Numeric profile (weighted means) — TEST\n", num_prof_te.round(3))

# categorical profile (top-3)
def wprop_table(cat_series, seg, w):
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

cat_to_show = [c for c in ["sex","education","class_of_worker","citizenship",
                           "region_of_previous_residence","age_bucket","weeks_bucket",
                           "is_union","is_self_employed","has_any_capital"]
               if c in Xte_feat.columns]
print("\n[HDBSCAN] Categorical profile (top-3 per segment) — TEST")
for c in cat_to_show:
    tab = wprop_table(Xte_feat[c], labels_test, w_te)
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((100 * top3).round(1))

# internal quality on TEST
from sklearn.metrics import silhouette_score
print("\n[HDBSCAN] silhouette_test:", silhouette_score(Z_te, labels_test))

# optional supervised lift on TEST (does not influence selection)
if 'y_test' in globals():
    pos_rate = ((w_te * y_test).groupby(labels_test).sum() / w_te.groupby(labels_test).sum())
    overall  = float((w_te * y_test).sum() / w_te.sum())
    lift     = (pos_rate / overall).rename("lift")
    pr_table = pd.DataFrame({
        "weighted_share_%": weighted_share_te.round(1),
        "pos_rate_%": (100*pos_rate).round(1),
        "lift": lift.round(2),
        "avg_strength": avg_strength.round(3),
    }).sort_values("lift", ascending=False)
    print("\n[HDBSCAN] Segment performance on TEST (weighted)\n", pr_table)

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


[HDBSCAN|trainval tuning] top-5
     mcs  min_samples method  sil_tv  noise_tv  min_share_tv  valid  score
6    600          NaN    eom   0.276     0.250         0.015   True  0.171
10   600          5.0    eom   0.329     0.078         0.011   True  0.141
8    600          1.0    eom   0.329     0.068         0.011   True  0.131
11   600          5.0   leaf   0.164     0.190         0.012   True  0.121
14  1200          1.0    eom   0.340     0.047         0.022   True  0.119

[Chosen params] {'min_cluster_size': 600, 'min_samples': None, 'cluster_selection_method': 'eom'}


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



[HDBSCAN] weighted_share TEST (%)
 segment
-1     25.0
 0      2.0
 1      1.1
 2      3.3
 3      4.2
 4      7.2
 5      3.6
 6      1.2
 7      2.6
 8      4.8
 9      2.6
 10    16.4
 11     9.5
 12    10.2
 13     1.3
 14     1.8
 15     1.8
 16     1.5
Name: weight, dtype: float64

[HDBSCAN] avg membership strength by segment (TEST)
 segment
-1     0.000
 0     0.931
 1     0.994
 2     0.840
 3     0.863
 4     0.966
 5     0.887
 6     0.998
 7     0.886
 8     0.576
 9     0.960
 10    0.819
 11    0.958
 12    0.960
 13    0.749
 14    0.931
 15    0.980
 16    0.866
Name: strength, dtype: float64


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[HDBSCAN] Numeric profile (weighted means) — TEST
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
-1       40.002                21.880              0.209     0.155             0.207    0.376
 0       43.438                42.087              0.425     0.075             0.707    0.123
 1       41.030                49.109              4.968     0.021             0.828    0.003
 2       48.817                39.038              0.227     0.044             0.665    0.182
 3       34.646                44.240              6.669     0.082             0.709    0.027
 4       50.417                31.036              0.000     0.000             0.533    0.340
 5       43.678                47.562              0.000     0.000             0.792    0.004
 6       29.851                51.288              0.000     1.000             0.964    0.000
 7      

# GMM

In [ ]:
# === Gaussian Mixture Models (soft clustering; choose K by weighted BIC) ===

# 0) 30-dim embeddings from your prep_pipe (dense)
Z_trainval = prep_pipe.fit_transform(X_trainval)
Z_test     = prep_pipe.transform(X_test)

# ---- helpers ---------------------------------------------------------------
def weighted_bic(gmm, Z, w):
    """Compute a BIC-like criterion with sample weights.
       BIC_w = -2 * sum_i w_i * log p(x_i) + p * log(sum_i w_i)
    """
    # per-sample log-likelihood
    ll = gmm.score_samples(Z)                   # shape (n,)
    n_eff = float(w.sum())

    # number of free parameters for GMM
    k = gmm.n_components
    d = Z.shape[1]
    cov = gmm.covariance_type
    if cov == "full":
        cov_params = k * d * (d + 1) // 2
    elif cov == "diag":
        cov_params = k * d
    elif cov == "tied":
        cov_params = d * (d + 1) // 2
    elif cov == "spherical":
        cov_params = k
    else:
        raise ValueError(f"Unknown covariance_type: {cov}")
    n_params = k * d + cov_params + (k - 1)     # means + covs + weights(k-1)

    return -2.0 * float((w * ll).sum()) + n_params * np.log(n_eff)

def pps_resample(Z, w, n_sub=None, rng=None):
    """Probability-proportional-to-size resampling WITH replacement."""
    if rng is None:
        rng = np.random.default_rng(42)
    n = len(Z)
    if n_sub is None:
        n_sub = n                              # keep same size
    p = w / w.sum()
    idx = rng.choice(n, size=n_sub, replace=True, p=p)
    return Z[idx]

# ---- (A) fit-time PPS to approximate weights --------------------------------
Z_sub = pps_resample(Z_trainval, w_trainval.values, n_sub=len(Z_trainval))  # or a cap like min(n, 120000)

# ---- (B) sweep K by weighted BIC; tie-break by silhouette -------------------
Ks = range(4, 11)
results = []

for k in Ks:
    gmm = GaussianMixture(
        n_components=k,
        covariance_type="full",
        reg_covar=1e-6,
        random_state=42,
        max_iter=500,
        init_params="kmeans"
    )
    # fit WITHOUT sample_weight (not supported in your sklearn)
    gmm.fit(Z_sub)

    # evaluate on FULL data with weights: weighted BIC (lower is better)
    bic_w = weighted_bic(gmm, Z_trainval, w_trainval.values)

    # optional: silhouette on hard labels (unweighted heuristic)
    labels = gmm.predict(Z_trainval)
    try:
        sil = silhouette_score(Z_trainval, labels, metric="euclidean")
    except Exception:
        sil = np.nan

    results.append((k, bic_w, sil, gmm))

# pick best by weighted BIC, break ties by higher silhouette
results.sort(key=lambda t: (t[1], -np.nan_to_num(t[2], nan=-1e9)))
best_k, best_bic_w, best_sil, best_model = results[0]
print(f"[GMM] best_k={best_k} | weighted-BIC={best_bic_w:.1f} | silhouette={best_sil:.3f}")

# ---- (C) label & probability, profiles -------------------------------------
labels_trainval = pd.Series(best_model.predict(Z_trainval), index=X_trainval.index, name="segment")
proba_trainval  = pd.DataFrame(best_model.predict_proba(Z_trainval), index=X_trainval.index)
labels_test     = pd.Series(best_model.predict(Z_test), index=X_test.index, name="segment")
proba_test      = pd.DataFrame(best_model.predict_proba(Z_test), index=X_test.index)

core_mask = proba_trainval.max(axis=1) >= 0.7
print(f"[GMM] core members ratio: {core_mask.mean():.2%}")

wshare = 100 * w_trainval.groupby(labels_trainval).sum() / w_trainval.sum()
print("\n[GMM] weighted_share (%)\n", wshare.round(1))

Xtv_feat = prep_pipe.named_steps["feat"].transform(X_trainval.copy())

def wmean_by_seg(s_values, seg, w):
    num = (s_values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(s_values.name)

num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour","is_mover",
                           "full_year_worker","no_work"] if c in Xtv_feat.columns]
num_prof = pd.concat([wmean_by_seg(Xtv_feat[c], labels_trainval, w_trainval) for c in num_to_show], axis=1)
print("\n[GMM] Numeric profile (weighted means)\n", num_prof.round(3))

In [28]:
# === Test-set reporting for the chosen GMM ===
# Assumes you already have:
# - best_model (fitted GMM)
# - Z_test (SVD embeddings of X_test)
# - labels_test, proba_test from best_model on Z_test
# - prep_pipe (FeatureBuilder + preprocess + SVD), w_test, y_test (optional)

# 0) Build post-FeatureBuilder features for TEST (readable profiling)
Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

# 1) Helpers (weighted mean / weighted category %)
def wmean_by_seg(values, seg, w):
    """Weighted mean grouped by segment."""
    num = (values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(values.name)

def wprop_table(cat_series, seg, w):
    """Weighted proportion table: rows=segment, cols=category."""
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

# 2) Basic segment sizes and confidence on TEST
seg_te = labels_test.reindex(X_test.index)
w_te   = w_test.reindex(X_test.index)

weighted_share_te = 100.0 * w_te.groupby(seg_te).sum() / w_te.sum()
print("\n[GMM] weighted_share TEST (%)\n", weighted_share_te.round(1))

# confidence = max posterior prob per sample, averaged (weighted) per segment
max_prob_te = proba_test.max(axis=1).rename("max_prob")
avg_conf_te = wmean_by_seg(max_prob_te, seg_te, w_te)
print("\n[GMM] avg max-membership prob by segment (TEST)\n", avg_conf_te.round(3))

# 3) Numeric profile (weighted means) on TEST
num_to_show = [
    c for c in ["age","weeks_worked_in_year","log_wage_per_hour",
                "is_mover","full_year_worker","no_work"]
    if c in Xte_feat.columns
]
num_profiles_test = pd.concat(
    [wmean_by_seg(Xte_feat[c], seg_te, w_te) for c in num_to_show], axis=1
)
print("\n[GMM] Numeric profile (weighted means) — TEST\n", num_profiles_test.round(3))

# 4) Categorical profile (weighted %; top-3 per segment) on TEST
cat_to_show = [
    c for c in ["sex","education","class_of_worker","citizenship",
                "region_of_previous_residence","age_bucket","weeks_bucket",
                "is_union","is_self_employed","has_any_capital"]
    if c in Xte_feat.columns
]
print("\n[GMM] Categorical profile (top-3 per segment) — TEST")
for c in cat_to_show:
    tab = wprop_table(Xte_feat[c], seg_te, w_te)
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((100 * top3).round(1))

# 5) Optional: internal quality & supervised lift on TEST
from sklearn.metrics import silhouette_score
print("\n[GMM] silhouette_test:", silhouette_score(Z_test, seg_te))

if 'y_test' in globals():
    pos_rate = ((w_te * y_test).groupby(seg_te).sum() / w_te.groupby(seg_te).sum())
    overall  = float((w_te * y_test).sum() / w_te.sum())
    lift     = (pos_rate / overall).rename("lift")
    pr_table = pd.DataFrame({
        "weighted_share_%": weighted_share_te.round(1),
        "pos_rate_%": (100 * pos_rate).round(1),
        "lift": lift.round(2),
        "avg_conf": avg_conf_te.round(3),
    }).sort_values("lift", ascending=False)
    print("\n[GMM] Segment performance on TEST (weighted)\n", pr_table)

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[GMM] weighted_share TEST (%)
 segment
0     8.9
1    22.4
2    23.0
3    14.4
4     3.8
5    12.8
6     5.5
7     0.2
8     1.9
9     7.0
Name: weight, dtype: float64

[GMM] avg max-membership prob by segment (TEST)
 segment
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: max_prob, dtype: float64

[GMM] Numeric profile (weighted means) — TEST
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
0        49.571                33.293              0.000     0.047             0.561    0.287
1         8.170                 0.000              0.000     0.000             0.000    1.000
2        39.467                51.922              0.000     0.000             1.000    0.000
3        60.819                 0.000              0.000     0.000             0.000    1.000
4        48.348                39.849   

In [ ]:


# -- 0) Build 30-D embeddings (reuse your prep_pipe)
Z_trainval = prep_pipe.fit_transform(X_trainval)
Z_test     = prep_pipe.transform(X_test)

# -- 1) Helpers --------------------------------------------------------------
def weighted_bic(gmm, Z, w):
    """BIC-like criterion with sample weights:
       BIC_w = -2 * sum_i w_i * log p(x_i) + p * log(sum_i w_i)
    """
    ll = gmm.score_samples(Z)  # (n,)
    n_eff = float(w.sum())

    k = gmm.n_components
    d = Z.shape[1]
    cov = gmm.covariance_type
    if cov == "full":
        cov_params = k * d * (d + 1) // 2
    elif cov == "diag":
        cov_params = k * d
    elif cov == "tied":
        cov_params = d * (d + 1) // 2
    elif cov == "spherical":
        cov_params = k
    else:
        raise ValueError(f"Unknown covariance_type: {cov}")
    n_params = k * d + cov_params + (k - 1)  # means + covs + weights(k-1)

    return -2.0 * float((w * ll).sum()) + n_params * np.log(n_eff)

def pps_resample(Z, w, n_sub=None, rng=None):
    """Probability-proportional-to-size resampling (with replacement)."""
    if rng is None:
        rng = np.random.default_rng(42)
    n = len(Z)
    if n_sub is None:
        n_sub = n
    p = np.asarray(w).reshape(-1)
    p = p / p.sum()
    idx = rng.choice(n, size=n_sub, replace=True, p=p)
    return Z[idx]

# -- 2) Tiny tuning sweep (fast) --------------------------------------------
rng = np.random.default_rng(123)
# cap the subsample to speed up EM; enlarge if you want a bit more accuracy
n_sub_cap = min(len(Z_trainval), 80000)
Z_sub = pps_resample(Z_trainval, w_trainval.values, n_sub=n_sub_cap, rng=rng)

# small grid centered around typical good values; adjust if you like
k_coarse   = [6, 8, 10, 12]            # coarse K
cov_types  = ["full", "diag"]          # two covariance structures
reg_list   = [1e-6, 1e-5]              # small regularization
seeds      = [17, 23]                  # 2 seeds to keep it fast
max_iter   = 300

cands = []
for k in k_coarse:
    for cov in cov_types:
        for reg in reg_list:
            for sd in seeds:
                gmm = GaussianMixture(
                    n_components=k,
                    covariance_type=cov,
                    reg_covar=reg,
                    random_state=int(sd),
                    max_iter=max_iter,
                    init_params="kmeans",
                    n_init=1,  # keep it fast
                )
                gmm.fit(Z_sub)  # no sample_weight in your sklearn
                bic_w = weighted_bic(gmm, Z_trainval, w_trainval.values)
                try:
                    sil = silhouette_score(Z_trainval, gmm.predict(Z_trainval), metric="euclidean")
                except Exception:
                    sil = np.nan
                cands.append({
                    "k": k, "cov": cov, "reg": reg, "seed": int(sd),
                    "bic_w": float(bic_w), "sil": float(sil), "model": gmm
                })

# pick top-3 by BIC_w (lower better), break ties by higher silhouette
cands.sort(key=lambda d: (d["bic_w"], -np.nan_to_num(d["sil"], nan=-1e9)))
top = cands[:3]

# quick local refine around top K values (±1), one seed to save time
k_refine = sorted(set(sum(([max(2, t["k"]-1), t["k"]+1] for t in top), [])))
for k in k_refine:
    for cov in cov_types:
        for reg in reg_list:
            gmm = GaussianMixture(
                n_components=k,
                covariance_type=cov,
                reg_covar=reg,
                random_state=42,
                max_iter=max_iter,
                init_params="kmeans",
                n_init=1,
            )
            gmm.fit(Z_sub)
            bic_w = weighted_bic(gmm, Z_trainval, w_trainval.values)
            try:
                sil = silhouette_score(Z_trainval, gmm.predict(Z_trainval), metric="euclidean")
            except Exception:
                sil = np.nan
            cands.append({
                "k": k, "cov": cov, "reg": reg, "seed": 42,
                "bic_w": float(bic_w), "sil": float(sil), "model": gmm
            })

# final pick
cands.sort(key=lambda d: (d["bic_w"], -np.nan_to_num(d["sil"], nan=-1e9)))
best = cands[0]
print(f"[GMM quick-tune] best: k={best['k']} | cov={best['cov']} | reg={best['reg']} "
      f"| seed={best['seed']} | BIC_w={best['bic_w']:.1f} | sil={best['sil']:.3f}")

# -- 3) Refit the best hyperparams on FULL TrainVal (one final fit) ----------
best_model = GaussianMixture(
    n_components=best["k"],
    covariance_type=best["cov"],
    reg_covar=best["reg"],
    random_state=best["seed"],
    max_iter=max_iter,
    init_params="kmeans",
    n_init=1,
)
best_model.fit(Z_trainval)

# label & proba on TEST
labels_test = pd.Series(best_model.predict(Z_test), index=X_test.index, name="segment")
proba_test  = pd.DataFrame(best_model.predict_proba(Z_test), index=X_test.index)

# -- 4) Final TEST report (same style as you used) ---------------------------
# readable TEST features (post-FeatureBuilder)
Xte_feat = prep_pipe.named_steps["feat"].transform(X_test.copy())

def wmean_by_seg(values, seg, w):
    """Weighted mean grouped by segment."""
    num = (values * w).groupby(seg).sum()
    den = w.groupby(seg).sum()
    return (num / den).rename(values.name)

def wprop_table(cat_series, seg, w):
    """Weighted proportion table: rows=segment, cols=category."""
    dummies = pd.get_dummies(cat_series.astype(str), dummy_na=False)
    wmat = dummies.mul(w.values.reshape(-1, 1))
    grp = wmat.groupby(seg).sum()
    grp = grp.div(grp.sum(axis=1), axis=0)
    grp.columns.name = cat_series.name
    return grp

# sizes & confidence
seg_te = labels_test.reindex(X_test.index)
w_te   = w_test.reindex(X_test.index)
weighted_share_te = 100.0 * w_te.groupby(seg_te).sum() / w_te.sum()
print("\n[GMM] weighted_share TEST (%)\n", weighted_share_te.round(1))

max_prob_te = proba_test.max(axis=1).rename("max_prob")
avg_conf_te = wmean_by_seg(max_prob_te, seg_te, w_te)
print("\n[GMM] avg max-membership prob by segment (TEST)\n", avg_conf_te.round(3))

# numeric profile
num_to_show = [c for c in ["age","weeks_worked_in_year","log_wage_per_hour",
                           "is_mover","full_year_worker","no_work"]
               if c in Xte_feat.columns]
num_profiles_test = pd.concat([wmean_by_seg(Xte_feat[c], seg_te, w_te) for c in num_to_show], axis=1)
print("\n[GMM] Numeric profile (weighted means) — TEST\n", num_profiles_test.round(3))

# categorical profile (top-3)
cat_to_show = [c for c in ["sex","education","class_of_worker","citizenship",
                           "region_of_previous_residence","age_bucket","weeks_bucket",
                           "is_union","is_self_employed","has_any_capital"]
               if c in Xte_feat.columns]
print("\n[GMM] Categorical profile (top-3 per segment) — TEST")
for c in cat_to_show:
    tab = wprop_table(Xte_feat[c], seg_te, w_te)
    top3 = tab.apply(lambda r: r.sort_values(ascending=False).head(3), axis=1)
    print(f"\n- {c} (weighted %):")
    print((100 * top3).round(1))

# internal quality & supervised lift (if y_test is available)
from sklearn.metrics import silhouette_score
print("\n[GMM] silhouette_test:", silhouette_score(Z_test, seg_te))

if 'y_test' in globals():
    pos_rate = ((w_te * y_test).groupby(seg_te).sum() / w_te.groupby(seg_te).sum())
    overall  = float((w_te * y_test).sum() / w_te.sum())
    lift     = (pos_rate / overall).rename("lift")
    pr_table = pd.DataFrame({
        "weighted_share_%": weighted_share_te.round(1),
        "pos_rate_%": (100 * pos_rate).round(1),
        "lift": lift.round(2),
        "avg_conf": avg_conf_te.round(3),
    }).sort_values("lift", ascending=False)
    print("\n[GMM] Segment performance on TEST (weighted)\n", pr_table)

/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


[GMM quick-tune] best: k=13 | cov=full | reg=1e-06 | seed=42 | BIC_w=-16504827.9 | sil=0.326


/Users/zxzhang/bertopic_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)



[GMM] weighted_share TEST (%)
 segment
0      7.3
1      4.7
2     22.4
3      3.4
4      8.8
5      2.0
6     14.3
7      0.3
8     12.6
9      6.7
10    11.8
11     4.2
12     1.4
Name: weight, dtype: float64

[GMM] avg max-membership prob by segment (TEST)
 segment
0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
Name: max_prob, dtype: float64

[GMM] Numeric profile (weighted means) — TEST
             age  weeks_worked_in_year  log_wage_per_hour  is_mover  full_year_worker  no_work
segment                                                                                      
0        26.982                51.910              0.000     0.000             1.000    0.000
1        35.410                44.037              6.689     0.106             0.707    0.031
2         8.170                 0.000              0.000     0.000             0.000    1.000
3        48.811                39.090              

# Final result


In [ ]:
# Z_trainval / Z_test: 30-dim SVD features from your prep_pipe
try:
    Z_trainval, Z_test  # noqa
except NameError:
    Z_trainval = prep_pipe.fit_transform(X_trainval)
    Z_test     = prep_pipe.transform(X_test)

# ------------------------------------------------------------------
# KMEANS LABELS
# ------------------------------------------------------------------
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

def _safe_kmeans(k, seed=42):
    try:
        return KMeans(n_clusters=k, n_init="auto", random_state=seed)
    except TypeError:
        return KMeans(n_clusters=k, n_init=10, random_state=seed)

# If you already have a fitted kmeans, reuse it; otherwise fit quickly with your chosen best_k (e.g., 10)
best_k_km = 10  # <- set this to the K you selected earlier
if "kmeans" not in globals() or not hasattr(kmeans, "labels_") or len(getattr(kmeans, "labels_", [])) != len(Z_trainval):
    kmeans = _safe_kmeans(best_k_km, seed=42)
    kmeans.fit(Z_trainval, sample_weight=w_trainval)

labels_trainval_km = pd.Series(kmeans.labels_, index=X_trainval.index, name="segment").astype(int)
labels_test_km     = pd.Series(kmeans.predict(Z_test), index=X_test.index, name="segment").astype(int)

# ------------------------------------------------------------------
# HDBSCAN LABELS (fit on PPS subsample; assign full set via approximate_predict)
# ------------------------------------------------------------------
import hdbscan

def _pps_subsample(Z, w, cap=60000, seed=42):
    rng = np.random.default_rng(seed)
    n = len(Z)
    n_sub = min(n, cap)
    p = (w.values / w.values.sum())
    idx = rng.choice(n, size=n_sub, replace=False, p=p)
    return Z[idx], n_sub

if "clusterer" not in globals() or not isinstance(clusterer, hdbscan.HDBSCAN):
    Z_sub, n_sub = _pps_subsample(Z_trainval, w_trainval, cap=60000, seed=42)
    min_cluster_size = max(200, int(0.01 * n_sub))  # tweak if you want fewer noise points
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=None,
        metric="euclidean",
        cluster_selection_method="eom",
        approx_min_span_tree=True,
        prediction_data=True
    )
    clusterer.fit(Z_sub)

tr_labels_hdb, tr_strength_hdb = hdbscan.approximate_predict(clusterer, Z_trainval)
te_labels_hdb, te_strength_hdb = hdbscan.approximate_predict(clusterer, Z_test)

labels_trainval_hdb = pd.Series(tr_labels_hdb, index=X_trainval.index, name="segment").astype(int)
labels_test_hdb     = pd.Series(te_labels_hdb, index=X_test.index, name="segment").astype(int)
# strength arrays (float) aligned to the same indices:
tr_strength_hdb = pd.Series(tr_strength_hdb, index=X_trainval.index, name="strength")
te_strength_hdb = pd.Series(te_strength_hdb, index=X_test.index, name="strength")

# ------------------------------------------------------------------
# GMM LABELS (quick refit if needed; use your best_k from weighted-BIC search)
# ------------------------------------------------------------------
from sklearn.mixture import GaussianMixture

best_k_gmm = 10  # <- set this to the GMM best_k you found (your prints show 10 clusters)
def _fit_gmm_quick(Z, n_comp=10, seed=42):
    g = GaussianMixture(
        n_components=n_comp,
        covariance_type="full",   # try "diag" if posteriors look too hard
        reg_covar=1e-3,           # a bit larger to avoid overly hard assignments
        max_iter=500,
        random_state=seed,
        init_params="kmeans",
    )
    g.fit(Z)
    return g

if "best_model" in globals() and isinstance(best_model, GaussianMixture) and best_model.n_components == best_k_gmm:
    gmm = best_model
else:
    gmm = _fit_gmm_quick(Z_trainval, n_comp=best_k_gmm, seed=42)

labels_trainval_gmm = pd.Series(gmm.predict(Z_trainval), index=X_trainval.index, name="segment").astype(int)
labels_test_gmm     = pd.Series(gmm.predict(Z_test),     index=X_test.index,     name="segment").astype(int)

# also keep soft probabilities (can be useful for scorecards / calibration)
proba_trainval_gmm  = pd.DataFrame(gmm.predict_proba(Z_trainval), index=X_trainval.index)
proba_test_gmm      = pd.DataFrame(gmm.predict_proba(Z_test),     index=X_test.index)

# ------------------------------------------------------------------
# (Optional) quick sanity checks
# ------------------------------------------------------------------
def _wshare(lbls, w):
    s = (w.groupby(lbls).sum() / w.sum()) * 100.0
    return s.sort_index().round(1)

print("[Sanity] KMeans Test weighted share %:\n", _wshare(labels_test_km, w_test))
print("[Sanity] HDBSCAN Test weighted share % (incl. -1 noise):\n", _wshare(labels_test_hdb, w_test))
print("[Sanity] GMM Test weighted share %:\n", _wshare(labels_test_gmm, w_test))# === Scorecard to compare KMeans / HDBSCAN / GMM on TEST ===


[Sanity] KMeans Test weighted share %:
 segment
0    17.6
1    34.5
2     0.3
3     5.5
4     3.5
5     2.0
6     8.9
7    27.6
Name: weight, dtype: float64
[Sanity] HDBSCAN Test weighted share % (incl. -1 noise):
 segment
-1     25.2
 0      2.0
 1      3.4
 2      1.1
 3      4.1
 4      7.1
 5      3.6
 6      1.2
 7      2.5
 8      2.6
 9      4.8
 10    16.3
 11     9.5
 12    10.2
 13     1.3
 14     1.8
 15     1.5
 16     1.8
Name: weight, dtype: float64
[Sanity] GMM Test weighted share %:
 segment
0     8.9
1    22.4
2    23.0
3    14.4
4     3.8
5    12.8
6     5.5
7     0.2
8     1.9
9     7.0
Name: weight, dtype: float64


In [ ]:
# === Scorecard to compare KMeans / HDBSCAN / GMM on TEST ===



# --- Inputs you should provide ---
# labels_trainval_km, labels_test_km
# labels_trainval_hdb, labels_test_hdb
# labels_trainval_gmm, labels_test_gmm
# Z_trainval, Z_test  (the SVD embeddings)
# w_trainval, w_test  (pd.Series aligned to indices)
# y_trainval, y_test  (pd.Series; optional but recommended)

methods = {
    "kmeans":  {"tr": labels_trainval_km,  "te": labels_test_km},
    "hdbscan": {"tr": labels_trainval_hdb, "te": labels_test_hdb},
    "gmm":     {"tr": labels_trainval_gmm, "te": labels_test_gmm},
}

def weighted_share(labels, w):
    return (w.groupby(labels).sum() / w.sum()).sort_index()

def share_drift_L1(s_tv, s_te):
    # Align indexes and compute L1 drift in percentage points
    idx = s_tv.index.union(s_te.index)
    a = s_tv.reindex(idx, fill_value=0.0)
    b = s_te.reindex(idx, fill_value=0.0)
    return float((a - b).abs().sum() * 100)

def topX_capture(labels, y, w, top_share=0.30):
    """Sort clusters by weighted pos rate desc; take clusters until covering top_share of population.
       Return (pop_covered%, positives_captured%) on TEST."""
    grp_w = w.groupby(labels).sum()
    pos_w = (w * y).groupby(labels).sum()
    pos_rate = (pos_w / grp_w).fillna(0.0)
    order = pos_rate.sort_values(ascending=False).index

    cum_pop = 0.0
    cum_pos = 0.0
    total_pop = float(w.sum())
    total_pos = float((w * y).sum())
    covered = []
    for lab in order:
        share = float(grp_w.loc[lab] / total_pop)
        covered.append(lab)
        cum_pop += share
        cum_pos += float(pos_w.loc[lab] / total_pos if total_pos > 0 else 0.0)
        if cum_pop >= top_share:
            break
    return 100*cum_pop, 100*cum_pos, pos_rate.loc[covered]

def weighted_mutual_info(labels, y, w):
    """Approximate weighted MI by PPS upsampling (fast & simple)."""
    # Draw n samples with probability proportional to w
    n = len(w)
    p = (w / w.sum()).values
    rng = np.random.default_rng(42)
    idx = rng.choice(np.arange(n), size=min(n, 200_000), replace=True, p=p)
    return mutual_info_score(labels.values[idx], y.values[idx])

rows = []
for name, d in methods.items():
    lab_tr, lab_te = d["tr"], d["te"]

    # shares
    s_tr = weighted_share(lab_tr, w_trainval)
    s_te = weighted_share(lab_te, w_test)
    drift = share_drift_L1(s_tr, s_te)

    # cluster counts (exclude noise = -1 for HDBSCAN when reporting)
    n_all = lab_te.nunique()
    n_eff = int((s_te[s_te >= 0.02]).count())  # clusters with >=2% weighted share
    noise_share = float((s_te.loc[-1]*100) if (-1 in s_te.index) else 0.0)

    # internal quality (silhouette on embeddings; sklearn has no weighted version)
    try:
        sil_tr = float(silhouette_score(Z_trainval, lab_tr))
        sil_te = float(silhouette_score(Z_test, lab_te))
    except Exception:
        sil_tr = np.nan; sil_te = np.nan

    # supervised utility (if y is available)
    if 'y_test' in globals():
        pop_cov, pos_cap, top_list = topX_capture(lab_te, y_test, w_test, top_share=0.30)
        mi = weighted_mutual_info(lab_te, y_test, w_test)
    else:
        pop_cov = pos_cap = mi = np.nan

    rows.append({
        "model": name,
        "clusters_all": n_all,
        "clusters_>=2%": n_eff,
        "noise_%": round(noise_share, 1),
        "sil_train": round(sil_tr, 3) if not np.isnan(sil_tr) else np.nan,
        "sil_test":  round(sil_te, 3) if not np.isnan(sil_te) else np.nan,
        "share_drift_pp": round(drift, 1),
        "Top30_pop_%": round(pop_cov, 1),
        "Top30_pos_%": round(pos_cap, 1),
        "MI_seg_y": round(mi, 4) if not np.isnan(mi) else np.nan,
    })

scorecard = pd.DataFrame(rows).set_index("model").sort_values("Top30_pos_%", ascending=False)
print(scorecard)

         clusters_all  clusters_>=2%  noise_%  sil_train  sil_test  share_drift_pp  Top30_pop_%  Top30_pos_%  MI_seg_y
model                                                                                                                 
kmeans              8              7      0.0      0.323     0.320             0.7         49.2         94.9    0.0533
gmm                10              8      0.0      0.322     0.319             1.0         37.8         90.3    0.0553
hdbscan            18             11     25.2      0.276     0.270             1.6         32.4         74.4    0.0436


In [ ]:
# ===============================================================
# Segmentation - Finalized (KMeans fixed, HDBSCAN/GMM = tuned)
# ===============================================================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, mutual_info_score

# -----------------------------
# 0) Embeddings (prep_pipe SVD)
# -----------------------------
try:
    Z_trainval, Z_test  # noqa
except NameError:
    Z_trainval = prep_pipe.fit_transform(X_trainval)
    Z_test     = prep_pipe.transform(X_test)

# ------------------------------------------------------
# 1) Tuned params (from your completed tuning results)
# ------------------------------------------------------
HDBSCAN_TUNED = dict(
    min_cluster_size=600,
    min_samples=None,                 # chosen in tuning
    cluster_selection_method="eom",
    metric="euclidean",
    approx_min_span_tree=True,
    prediction_data=True,
)

GMM_TUNED = dict(
    n_components=13,                  # tuned result; set to 10 if you prefer earlier 10-cluster solution
    covariance_type="full",
    reg_covar=1e-3,
    max_iter=500,
    random_state=42,
    init_params="kmeans",
)

# ------------------------------------------------------
# 2) KMEANS (keep as-is; already tuned on your side)
# ------------------------------------------------------
from sklearn.cluster import KMeans

def _safe_kmeans(k, seed=42):
    try:
        return KMeans(n_clusters=k, n_init="auto", random_state=seed)
    except TypeError:
        return KMeans(n_clusters=k, n_init=10, random_state=seed)

best_k_km = 10  # your tuned K

if "kmeans" not in globals() or not hasattr(kmeans, "labels_") or len(getattr(kmeans, "labels_", [])) != len(Z_trainval):
    kmeans = _safe_kmeans(best_k_km, seed=42)
   
    try:
        kmeans.fit(Z_trainval, sample_weight=w_trainval)
    except TypeError:
        kmeans.fit(Z_trainval)

labels_trainval_km = pd.Series(kmeans.labels_, index=X_trainval.index, name="segment").astype(int)
labels_test_km     = pd.Series(kmeans.predict(Z_test), index=X_test.index, name="segment").astype(int)

# ------------------------------------------------------
# 3) HDBSCAN (use PPS subsample + tuned params)
# ------------------------------------------------------
import hdbscan

def _pps_subsample(Z, w, cap=60000, seed=42):
    rng = np.random.default_rng(seed)
    n = len(Z)
    n_sub = min(n, cap)
    p = (w.values / w.values.sum())
    idx = rng.choice(n, size=n_sub, replace=False, p=p)
    return Z[idx]

def _need_refit_hdbscan(clf, tuned):
    if "clusterer" not in globals(): return True
    if not isinstance(clf, hdbscan.HDBSCAN): return True
    return not (
        clf.min_cluster_size == tuned["min_cluster_size"] and
        clf.min_samples == tuned["min_samples"] and
        clf.cluster_selection_method == tuned["cluster_selection_method"] and
        clf.metric == tuned["metric"]
    )

if _need_refit_hdbscan(globals().get("clusterer", None), HDBSCAN_TUNED):
    Z_sub = _pps_subsample(Z_trainval, w_trainval, cap=60000, seed=42)
    clusterer = hdbscan.HDBSCAN(**HDBSCAN_TUNED)
    clusterer.fit(Z_sub)

tr_labels_hdb, tr_strength_arr = hdbscan.approximate_predict(clusterer, Z_trainval)
te_labels_hdb, te_strength_arr = hdbscan.approximate_predict(clusterer, Z_test)

labels_trainval_hdb = pd.Series(tr_labels_hdb, index=X_trainval.index, name="segment").astype(int)
labels_test_hdb     = pd.Series(te_labels_hdb, index=X_test.index,     name="segment").astype(int)
tr_strength_hdb     = pd.Series(tr_strength_arr, index=X_trainval.index, name="strength")
te_strength_hdb     = pd.Series(te_strength_arr, index=X_test.index,     name="strength")

# ------------------------------------------------------
# 4) GMM (use tuned params; refit if mismatch)
# ------------------------------------------------------
from sklearn.mixture import GaussianMixture

def _need_refit_gmm(g, tuned):
    if "gmm" not in globals(): return True
    if not isinstance(g, GaussianMixture): return True
    if g.n_components != tuned["n_components"]: return True
    if g.covariance_type != tuned["covariance_type"]: return True
    if not np.isclose(getattr(g, "reg_covar", tuned["reg_covar"]), tuned["reg_covar"]): return True
    return False

if _need_refit_gmm(globals().get("gmm", None), GMM_TUNED):
    gmm = GaussianMixture(**GMM_TUNED).fit(Z_trainval)

labels_trainval_gmm = pd.Series(gmm.predict(Z_trainval), index=X_trainval.index, name="segment").astype(int)
labels_test_gmm     = pd.Series(gmm.predict(Z_test),     index=X_test.index,     name="segment").astype(int)
proba_trainval_gmm  = pd.DataFrame(gmm.predict_proba(Z_trainval), index=X_trainval.index)
proba_test_gmm      = pd.DataFrame(gmm.predict_proba(Z_test),     index=X_test.index)

# ------------------------------------------------------
# 5) Sanity prints (weighted shares on TEST)
# ------------------------------------------------------
def _wshare(lbls, w):
    return (w.groupby(lbls).sum() / w.sum() * 100.0).sort_index().round(1)

print("[Sanity] KMeans | Test weighted share %\n", _wshare(labels_test_km,  w_test))
print("[Sanity] HDBSCAN | Test weighted share % (incl. -1 noise)\n", _wshare(labels_test_hdb, w_test))
print("[Sanity] GMM | Test weighted share %\n", _wshare(labels_test_gmm,  w_test))

# ------------------------------------------------------
# 6) Unified scorecard on TEST (and TRAINVAL for drift)
# ------------------------------------------------------
EXCLUDE_HDBSCAN_NOISE_IN_COUNTS = True  # set to False if you want to count noise as a cluster

methods = {
    "kmeans":  {"tr": labels_trainval_km,  "te": labels_test_km},
    "hdbscan": {"tr": labels_trainval_hdb, "te": labels_test_hdb},
    "gmm":     {"tr": labels_trainval_gmm, "te": labels_test_gmm},
}

def weighted_share(labels, w):
    return (w.groupby(labels).sum() / w.sum()).sort_index()

def share_drift_L1(s_tv, s_te):
    idx = s_tv.index.union(s_te.index)
    a = s_tv.reindex(idx, fill_value=0.0)
    b = s_te.reindex(idx, fill_value=0.0)
    return float((a - b).abs().sum() * 100)  # percentage points

def topX_capture(labels, y, w, top_share=0.30):
    grp_w = w.groupby(labels).sum()
    pos_w = (w * y).groupby(labels).sum()
    pos_rate = (pos_w / grp_w).fillna(0.0)
    order = pos_rate.sort_values(ascending=False).index

    cum_pop = 0.0
    cum_pos = 0.0
    total_pop = float(w.sum())
    total_pos = float((w * y).sum())
    covered = []
    for lab in order:
        share = float(grp_w.loc[lab] / total_pop)
        covered.append(lab)
        cum_pop += share
        cum_pos += float(pos_w.loc[lab] / total_pos) if total_pos > 0 else 0.0
        if cum_pop >= top_share:
            break
    return 100*cum_pop, 100*cum_pos, pos_rate.loc[covered]

def weighted_mutual_info(labels, y, w):
    n = len(w)
    p = (w / w.sum()).values
    rng = np.random.default_rng(42)
    idx = rng.choice(np.arange(n), size=min(n, 200_000), replace=True, p=p)
    return float(mutual_info_score(labels.values[idx], y.values[idx]))

rows = []
for name, d in methods.items():
    lab_tr, lab_te = d["tr"], d["te"]

    # shares
    s_tr = weighted_share(lab_tr, w_trainval)
    s_te = weighted_share(lab_te, w_test)
    drift = share_drift_L1(s_tr, s_te)

    # cluster counts
    if name == "hdbscan" and EXCLUDE_HDBSCAN_NOISE_IN_COUNTS:
        s_te_eff = s_te.drop(index=-1, errors="ignore")
        n_all = s_te_eff.index.nunique()
        n_eff = int((s_te_eff >= 0.02).sum())    # >=2% weighted share
        noise_share = float((s_te.get(-1, 0.0)) * 100.0)
    else:
        n_all = lab_te.nunique()
        n_eff = int((s_te >= 0.02).sum())
        noise_share = float((s_te.get(-1, 0.0)) * 100.0)

    # internal quality (unweighted silhouette on embeddings)
    try:
        sil_tr = float(silhouette_score(Z_trainval, lab_tr))
        sil_te = float(silhouette_score(Z_test, lab_te))
    except Exception:
        sil_tr = np.nan; sil_te = np.nan

    # supervised utility (optional)
    if 'y_test' in globals():
        pop_cov, pos_cap, _ = topX_capture(lab_te, y_test, w_test, top_share=0.30)
        mi = weighted_mutual_info(lab_te, y_test, w_test)
    else:
        pop_cov = pos_cap = mi = np.nan

    rows.append({
        "model": name,
        "clusters_all": n_all,
        "clusters_>=2%": n_eff,
        "noise_%": round(noise_share, 1),
        "sil_train": round(sil_tr, 3) if not np.isnan(sil_tr) else np.nan,
        "sil_test":  round(sil_te, 3) if not np.isnan(sil_te) else np.nan,
        "share_drift_pp": round(drift, 1),
        "Top30_pop_%": round(pop_cov, 1),
        "Top30_pos_%": round(pos_cap, 1),
        "MI_seg_y": round(mi, 4) if not np.isnan(mi) else np.nan,
    })

scorecard = pd.DataFrame(rows).set_index("model").sort_values(
    ["Top30_pos_%", "sil_test"], ascending=[False, False]
)
print("\n=== Scorecard (TEST) ===\n", scorecard)